# Validation of FS vs GT and Ours vs GT

In [1]:
import numpy as np
import nibabel as nib
from pathlib import Path

In [2]:
orig_file = "/home/lugges/Documents/MPI_BK/validation/me0801/me0801.nii"
fs_file = "/home/lugges/Documents/MPI_BK/validation/me0801/me0801_rca/me0801_wts15_reconall/mri/ribbon.nii"
ours_file = "/home/lugges/Documents/MPI_BK/validation/me0801/me0801_ours/me0801_mask.nii"
gt_file = "/home/lugges/Documents/MPI_BK/validation/me0801/me0801_seg/ribbon_Dez_19.nii"

In [3]:
# load the files
fs_obj = nib.load(fs_file)
ours_obj = nib.load(ours_file)
gt_obj = nib.load(gt_file)
orig_obj = nib.load(orig_file)

In [4]:
fs_obj.shape

(352, 352, 352)

In [5]:
ours_obj.shape

(384, 384, 384)

In [6]:
gt_obj.shape

(352, 352, 352)

In [7]:
orig_obj.shape

(352, 352, 256)

In [8]:
fs_data = fs_obj.get_fdata()
ours_data = ours_obj.get_fdata()
gt_data = gt_obj.get_fdata()

In [9]:
np.unique(ours_data)

array([0., 1., 2., 3., 4.])

In [10]:
np.unique(fs_data)

array([ 0.,  2.,  3., 41., 42.])

In [11]:
np.unique(gt_data)

array([ 0.,  2.,  3., 41., 42.])

**Labels:**
Ribbon Prediction:

Freesurfer:
0: BG
2: Left WM
3: Left GM
41: Right WM
42: Right GM

Ours:
0: BG
1: Left WM
2: Left GM
3: Right WM
4: Right GM

|          | FS | Ours |
|----------|----|------|
| BG       | 0  | 0    |
| Left WM  | 2  | 1    |
| Left GM  | 3  | 2    |
| Right WM | 41 | 3    |
| Right GM | 42 | 4    |

In [12]:
def ours_to_fs_labels(data):
    data = np.where(data == 4, 42, data)
    data = np.where(data == 3, 41, data)
    data = np.where(data == 2, 3, data)
    data = np.where(data == 1, 2, data)
    return data

In [13]:
ours_data = ours_to_fs_labels(ours_data)

In [14]:
ours_data.shape

(384, 384, 384)

In [15]:
ours_obj.affine

array([[   0.60000002,   -0.        ,   -0.        , -116.69263458],
       [  -0.        ,    0.60000002,   -0.        , -128.11448669],
       [   0.        ,    0.        ,    0.60000002, -106.5426178 ],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [16]:
from nilearn.image import resample_img

In [17]:
ours_res = resample_img(ours_obj, target_affine=gt_obj.affine, target_shape=gt_obj.shape)

/home/lugges/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:531: UserWarning: Casting data from int8 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


In [18]:
ours_res.shape

(352, 352, 352)

In [19]:
ours_data = ours_to_fs_labels(ours_res.get_fdata())

In [20]:
np.unique(ours_data)

array([0.00000000e+00, 1.40129846e-45, 2.80259693e-45, ...,
       3.99999976e+00, 4.10000000e+01, 4.20000000e+01])

In [21]:
np.unique(ours_res.get_fdata())

array([0.00000000e+00, 1.40129846e-45, 2.80259693e-45, ...,
       3.99999928e+00, 3.99999976e+00, 4.00000000e+00])

In [22]:
# ToDo:
# transform ours.shape (384, 384, 384) to ours.shape(352, 352, 352)


# Metrics

- IoU, voraussichtlich min(IoU, boundary IoU)  # Drop Background
- Hausdorff Distance (voraussichtlich die 95. Quantile)
- Variation of Information



maybe create one channel for each label? -> transform to channel-wise binary labels?

meanIoU in pyTorch: https://github.com/wolny/pytorch-3dunet/blob/master/pytorch3dunet/unet3d/metrics.py



## Volumetric IoU

$$JAC = \frac{TP}{TP + FP + FN}$$

In [23]:
def volumetric_iou(gt, pred):
    ious = []
    for i in np.unique(gt):
        intersec = np.logical_and(gt==i, pred==i)
        union = np.logical_or(gt==i, pred==i)
        ious.append(np.sum(intersec)/np.sum(union))
    return sum(ious) / len(ious)

In [24]:
def hausdorff_distance():
    pass

## Variation of Information

$S_g$ is the ground truth partition and $S_t$ is the segmentation. Then the variation of information $VOI(S_g, S_t)$ is defined as:
$$VOI(S_g, S_t) = H(S_g) + H(S_t) - 2MI(S_g, S_t) $$
, where the mutual information $MI(S_g, S_t)$ is defined as:
$$MI(S_g, S_t) = H(S_g) + H(S_t) - H(S_g, S_t)$$

In [26]:
from scipy.stats import entropy

def mutual_information(gt, pred):
    pass


def variation_of_information(gt, pred):
    pass

In [27]:
def label_to_bool_arr(data, label):
    return np.where(data==label, 1, data)

In [28]:
test1 = label_to_bool_arr(gt_data, 2)

In [29]:
np.unique(test1)

array([ 0.,  1.,  3., 41., 42.])

In [30]:
hero = np.logical_and(gt_data==2, fs_data==2)

In [31]:
sucker = np.logical_or(gt_data==2, fs_data==2)

In [32]:
np.sum(hero) / np.sum(sucker)

0.9970613609692579

In [33]:
iou = volumetric_iou(gt_data, fs_data)

In [34]:
iou

0.9756320835680906